In [1]:
!git clone https://github.com/hila-chefer/Transformer-Explainability.git

import os
os.chdir(f'./Transformer-Explainability')

!pip install -r requirements.txt
!pip install captum

fatal: destination path 'Transformer-Explainability' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from transformers import AutoTokenizer

from captum.attr import (
    visualization
)
import torch

In [3]:
model = BertForSequenceClassification.from_pretrained("Willy/bert-base-spanish-wwm-cased-finetuned-NLP-IE-4").to("cuda")
model.eval()
tokenizer = AutoTokenizer.from_pretrained("Willy/bert-base-spanish-wwm-cased-finetuned-NLP-IE-4")
# initialize the explanations generator
explanations = Generator(model)

classifications = ["0", "1"]


# Class based

In [5]:
# encode a sentence
text_batch = ["Si voy a asistir"]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()

true_class = 1

target_class = 0
classification = output.argmax(dim=-1).item()

expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=11, index=target_class)[0]
expl = (expl - expl.min()) / (expl.max() - expl.min())

output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)

class_name = classifications[target_class]
if class_name == "1":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', 0.0), ('Si', 0.7684720754623413), ('voy', 0.15390169620513916), ('a', 0.13183704018592834), ('asistir', 1.0), ('[SEP]', 0.0)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (0.63),1,1.00,[CLS] Si voy a asistir [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (0.63),1,1.00,[CLS] Si voy a asistir [SEP]
